In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
print("water")

In [ ]:
dengue = pd.read_csv('DENGUE FEVER.csv')
dengue.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2811 entries, 0 to 2810
Data columns (total 30 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        2811 non-null   int64  
 1   year              2811 non-null   int64  
 2   week              2811 non-null   int64  
 3   month             2811 non-null   int64  
 4   adm3_pcode        2811 non-null   object 
 5   adm3_en           2811 non-null   object 
 6   tave              2811 non-null   float64
 7   tmin              2811 non-null   float64
 8   tmax              2811 non-null   float64
 9   heat_index        2811 non-null   float64
 10  pr                2811 non-null   float64
 11  wind_speed        2811 non-null   float64
 12  rh                2811 non-null   float64
 13  solar_rad         2811 non-null   float64
 14  uv_rad            2811 non-null   float64
 15  clinic_nearest    1686 non-null   float64
 16  dentist_nearest   1686 non-null   float64


In [ ]:
print(dengue.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2811 entries, 0 to 2810
Data columns (total 31 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        2811 non-null   int64  
 1   year              2811 non-null   int64  
 2   week              2811 non-null   int64  
 3   month             2811 non-null   int64  
 4   adm3_pcode        2811 non-null   object 
 5   adm3_en           2811 non-null   object 
 6   tave              2811 non-null   float64
 7   tmin              2811 non-null   float64
 8   tmax              2811 non-null   float64
 9   heat_index        2811 non-null   float64
 10  pr                2811 non-null   float64
 11  wind_speed        2811 non-null   float64
 12  rh                2811 non-null   float64
 13  solar_rad         2811 non-null   float64
 14  uv_rad            2811 non-null   float64
 15  clinic_nearest    1686 non-null   float64
 16  dentist_nearest   1686 non-null   float64


In [5]:
dengue_cleaned = dengue[dengue['year'] > 2014]
dengue_cleaned = dengue_cleaned.drop(['Unnamed: 0'], axis = 1)
dengue_cleaned.fillna(0, inplace = True)
dengue_cleaned = dengue_cleaned.apply(lambda x: x.map('{:.2f}'.format) if x.dtype == 'float64' else x)
adm3_pcode_mapping = {
    'PH137401000': 1,
    'PH137503000': 2,
    'PH137603000': 3
}
dengue_cleaned['adm3_pcode'] = dengue_cleaned['adm3_pcode'].map(adm3_pcode_mapping)
columns_to_drop = ['year', 'week', 'month', 'adm3_pcode', 'adm3_en', 'clinic_count', 'clinic_nearest']
dengue_cleaned = dengue_cleaned.drop(columns=columns_to_drop)

In [6]:
print(dengue_cleaned.info())

dengue_cleaned.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1500 entries, 1311 to 2810
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tave              1500 non-null   object
 1   tmin              1500 non-null   object
 2   tmax              1500 non-null   object
 3   heat_index        1500 non-null   object
 4   pr                1500 non-null   object
 5   wind_speed        1500 non-null   object
 6   rh                1500 non-null   object
 7   solar_rad         1500 non-null   object
 8   uv_rad            1500 non-null   object
 9   dentist_nearest   1500 non-null   object
 10  doctors_nearest   1500 non-null   object
 11  hospital_nearest  1500 non-null   object
 12  optician_nearest  1500 non-null   object
 13  pharmacy_nearest  1500 non-null   object
 14  pop_count_total   1500 non-null   object
 15  pop_density_mean  1500 non-null   object
 16  dentist_count     1500 non-null   object
 17  doctors_count   

,tave,tmin,tmax,heat_index,pr,wind_speed,rh,solar_rad,uv_rad,dentist_nearest,...,optician_nearest,pharmacy_nearest,pop_count_total,pop_density_mean,dentist_count,doctors_count,hospital_count,optician_count,pharmacy_count,case_total
1311,24.90,23.05,27.55,25.77,2.20,5.23,74.81,154.55,18.75,1461.16,...,1327.64,153.05,13551.19,35729.54,4.00,0.00,8.00,4.00,72.00,0.00
1312,24.67,22.73,27.29,25.38,0.00,4.65,72.92,158.34,19.29,10000.00,...,9998.30,949.48,20156.26,44536.11,0.00,0.00,0.00,0.00,4.00,0.00
1313,24.63,22.76,27.25,25.46,6.40,5.20,75.66,153.93,18.64,1800.19,...,1743.96,300.98,51784.80,11364.11,12.00,4.00,24.00,4.00,72.00,0.00
1314,25.38,23.15,28.33,26.46,0.00,3.72,74.96,178.14,21.64,1461.16,...,1327.64,153.05,13551.19,35729.54,7.00,0.00,14.00,7.00,126.00,10.00
1315,25.27,23.12,28.03,26.20,0.00,3.10,73.44,180.83,21.71,10000.00,...,9998.30,949.48,20156.26,44536.11,0.00,0.00,0.00,0.00,7.00,9.00


In [7]:
dengue_cleaned.to_csv('Cleaned_Dengue_Data.csv', index=False)

In [8]:
leptospirosis = pd.read_csv('LEPTOSPIROSIS.csv')
leptospirosis.head()
print(leptospirosis.info())

leptospirosis_cleaned = leptospirosis[leptospirosis['year'] > 2014]
leptospirosis_cleaned = leptospirosis_cleaned.drop(['Unnamed: 0'], axis = 1)
leptospirosis_cleaned.fillna(0, inplace = True)
leptospirosis_cleaned = leptospirosis_cleaned.apply(lambda x: x.map('{:.2f}'.format) if x.dtype == 'float64' else x)
adm3_pcode_mapping = {
    'PH137401000': 1,
    'PH137503000': 2,
    'PH137603000': 3
}
leptospirosis_cleaned['adm3_pcode'] = leptospirosis_cleaned['adm3_pcode'].map(adm3_pcode_mapping)
columns_to_drop_leptospirosis = ['year', 'week', 'month', 'adm3_pcode', 'adm3_en', 'clinic_count', 'clinic_nearest']
leptospirosis_cleaned = leptospirosis_cleaned.drop(columns=columns_to_drop_leptospirosis)

print(leptospirosis_cleaned.info())
leptospirosis_cleaned.head()

dengue_cleaned.to_csv('Cleaned_Leptospirosis_Data.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2811 entries, 0 to 2810
Data columns (total 30 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        2811 non-null   int64  
 1   year              2811 non-null   int64  
 2   week              2811 non-null   int64  
 3   month             2811 non-null   int64  
 4   adm3_pcode        2811 non-null   object 
 5   adm3_en           2811 non-null   object 
 6   tave              2811 non-null   float64
 7   tmin              2811 non-null   float64
 8   tmax              2811 non-null   float64
 9   heat_index        2811 non-null   float64
 10  pr                2811 non-null   float64
 11  wind_speed        2811 non-null   float64
 12  rh                2811 non-null   float64
 13  solar_rad         2811 non-null   float64
 14  uv_rad            2811 non-null   float64
 15  clinic_nearest    1686 non-null   float64
 16  dentist_nearest   1686 non-null   float64


In [9]:
typoid = pd.read_csv('TYPHOID FEVER.csv')
typoid.head()
print(typoid.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2811 entries, 0 to 2810
Data columns (total 30 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        2811 non-null   int64  
 1   year              2811 non-null   int64  
 2   week              2811 non-null   int64  
 3   month             2811 non-null   int64  
 4   adm3_pcode        2811 non-null   object 
 5   adm3_en           2811 non-null   object 
 6   tave              2811 non-null   float64
 7   tmin              2811 non-null   float64
 8   tmax              2811 non-null   float64
 9   heat_index        2811 non-null   float64
 10  pr                2811 non-null   float64
 11  wind_speed        2811 non-null   float64
 12  rh                2811 non-null   float64
 13  solar_rad         2811 non-null   float64
 14  uv_rad            2811 non-null   float64
 15  clinic_nearest    1686 non-null   float64
 16  dentist_nearest   1686 non-null   float64


In [10]:

typoid_cleaned = typoid[typoid['year'] > 2014]
typoid_cleaned = typoid_cleaned.drop(['Unnamed: 0'], axis = 1)
typoid_cleaned.fillna(0, inplace = True)
typoid_cleaned = typoid_cleaned.apply(lambda x: x.map('{:.2f}'.format) if x.dtype == 'float64' else x)
adm3_pcode_mapping = {
    'PH137401000': 1,
    'PH137503000': 2,
    'PH137603000': 3
}
typoid_cleaned['adm3_pcode'] = typoid_cleaned['adm3_pcode'].map(adm3_pcode_mapping)
columns_to_drop_typoid = ['year', 'week', 'month', 'adm3_pcode', 'adm3_en', 'clinic_count', 'clinic_nearest']
typoid_cleaned = typoid_cleaned.drop(columns=columns_to_drop_typoid)

print(typoid_cleaned.info())
typoid_cleaned.head()

dengue_cleaned.to_csv('Cleaned_Typoid_Data.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 1500 entries, 1311 to 2810
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tave              1500 non-null   object
 1   tmin              1500 non-null   object
 2   tmax              1500 non-null   object
 3   heat_index        1500 non-null   object
 4   pr                1500 non-null   object
 5   wind_speed        1500 non-null   object
 6   rh                1500 non-null   object
 7   solar_rad         1500 non-null   object
 8   uv_rad            1500 non-null   object
 9   dentist_nearest   1500 non-null   object
 10  doctors_nearest   1500 non-null   object
 11  hospital_nearest  1500 non-null   object
 12  optician_nearest  1500 non-null   object
 13  pharmacy_nearest  1500 non-null   object
 14  pop_count_total   1500 non-null   object
 15  pop_density_mean  1500 non-null   object
 16  dentist_count     1500 non-null   object
 17  doctors_count   